In [1]:
%matplotlib inline

import os,sys
import os.path as osp

import argparse
import os, tempfile, glob, shutil
import random
from datetime import datetime

import GPy
import GPyOpt
import subprocess as sub
import numpy as np
import scipy.spatial
import math
import matplotlib.pyplot as plt
from numpy import linalg as LA

from scipy import optimize
import time
import socket
from sklearn.neighbors import NearestNeighbors

global g_blender_executable_path,  blank_file, render_code, model_file, table_file, temp_dirname, epsilon, \
       filename, location_gt, rotation_gt, obj_start_config_array, hand_start_config_array, \
       obj_end_config_array, hand_end_config_array, id_of_sample, train_file, init_pose_file, final_pose_file

epsilon = 1e-6

if os.environ.get('PHYSICS_SIMULATOR_PATH') == None:
    print("Please set PHYSICS_SIMULATOR_PATH in bashrc!")
    sys.exit()

g_repo_path = os.environ['PHYSICS_SIMULATOR_PATH']

# Input Pose File path
init_pose_file = g_repo_path + '/tmp/init_pose.txt'
final_pose_file = g_repo_path + '/tmp/final_pose.txt'

In [3]:
def read_data(filename):
    count = 1
    obj_start_config_array = []
    hand_start_config_array = []
    obj_end_config_array = []
    hand_end_config_array = []
    
    with open(filename) as f:
        for line in f: # read rest of lines
            if (count-3) % 6 == 0: 
                hand_start_config = []
                hand_start_config.append([float(x) for x in line.split()])
                hand_start_config_a= np.array(hand_start_config)                    
                hand_start_config_array.append(hand_start_config_a)
            if (count-2) % 6 == 0: 
                obj_start_config = []
                obj_start_config.append([float(x) for x in line.split()])
                obj_start_config_a= np.array(obj_start_config)                    
                obj_start_config_array.append(obj_start_config_a)
            if (count-5) % 6 == 0: 
                hand_end_config = []
                hand_end_config.append([float(x) for x in line.split()])
                hand_end_config_a= np.array(hand_end_config)                    
                hand_end_config_array.append(hand_end_config_a)
            if (count-4) % 6 == 0: 
                obj_end_config = []
                obj_end_config.append([float(x) for x in line.split()])
                obj_end_config_a= np.array(obj_end_config)                    
                obj_end_config_array.append(obj_end_config_a)                    

            count += 1
    return obj_start_config_array, hand_start_config_array, obj_end_config_array, hand_end_config_array

In [4]:
def train(id, time_budget):
    print "starting exp ", id
    test_err =[]
    test_NN_err =[]
    
    ############ sample ##############
    sample_num = 1000
    
    start_time = time.time()
    elapsed_time = time.time() - start_time
    uncertainty = 0.04
    count = 1
#     print X.shape, Y.shape
    state_dim = X.shape[0]
    rbf = GPy.kern.RBF(state_dim, lengthscale=uncertainty, ARD=False)
    num_init_samples = 1
    X_train = np.zeros((num_init_samples, state_dim))
#     X_train[0,:] =[-uncertainty, +uncertainty]
#     X_train[1,:] =[uncertainty, -uncertainty]

    print "X_train: ", X_train
    Y_train = np.zeros((num_init_samples, state_dim))
    for i in range(0,num_init_samples):
        X_new = X_train[i,:]
#         print X_new.shape
        with open(init_pose_file, "w+") as file:
            file.write("%f %f \n" % ( X_new[0], X_new[1]))
         
        # Create a TCP/IP socket
        sock2 = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock2.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

        # Connect the socket to the port where the server is listening
        serv_address = ('localhost', 50008)
#         print('waiting for a connection')
        sock2.connect(serv_address)
#         print('Connected')
        sock2.sendall(str(id))
        sock2.close()   

        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        # Bind the socket to the port
        server_address = ('localhost', 10001)
        sock.bind(server_address)

        # Listen for incoming connections
        sock.listen(1)        
        connection, client_address = sock.accept()

        # Receive the data in small chunks and retransmit it
        data = connection.recv(16)
        connection.close()
        sock.close()
        
        
        Y_new = np.zeros((1, state_dim))

        with open(final_pose_file, "r") as file:
            for line in file:
                vals = line.split()
                for j in xrange(0,state_dim):
                    Y_new[0,i] = vals[j]
        
        print Y_new, i
        Y_train[i,0] = Y_new[0,0]
        Y_train[i,1] = Y_new[0,1]


    model = GPy.models.GPRegression(X_train, Y_train, kernel=rbf)
    model.randomize()
    model.optimize()
    

    while elapsed_time < time_budget:
#         X_sample = np.zeros((sample_num, state_dim))    
#         for i in xrange(0, state_dim):
#             X_sample[:, i] = np.random.uniform(-uncertainty, +uncertainty, sample_num)
        X_sample = np.random.uniform(-uncertainty, +uncertainty, (sample_num, state_dim))
#         origin = np.zeros((sample_num, state_dim))
#         sample_weight = np.power(e, -np.linalg.norm(X_sample-origin, axis=1)/np.sqrt(2*uncertainty**2))
        
#         print 'sample_weight', sample_weight.shape,sample_weight[0]
        (Y_predict, Y_var) = model.predict_noiseless(X_sample)
#         print Y_var.shape
        
#         Y_var = np.multiply(Y_var[:,0],sample_weight)
#         print Y_var.shape
#         plt.plot(Y_var)
#         raw_input("Press Enter to continue...")

    
    
        max_var = Y_var.argmax(axis = 0)
        X_new = X_sample[max_var,:]
#         print max_var
#         print Y_var[max_var]
        print 'new sample: ',X_new
        X_new_1 = X_new #np.zeros((1,2))
#         X_new_1[0,0] = X_new[0]
#         X_new_1[0,1] = X_new[1]

        X_train = np.concatenate((X_train, X_new_1),axis=0)
#         print X_train
        
        with open(init_pose_file, "w+") as file:
            file.write("%f %f \n" % ( X_new_1[0,0], X_new_1[0,1]))
         
        # Create a TCP/IP socket
        sock2 = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock2.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

        # Connect the socket to the port where the server is listening
        serv_address = ('localhost', 50008)
#         print('waiting for a connection')
        sock2.connect(serv_address)
#         print('Connected')
        sock2.sendall(str(id))
        sock2.close()   

        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        # Bind the socket to the port
        server_address = ('localhost', 10001)
        sock.bind(server_address)

        # Listen for incoming connections
        sock.listen(1)        
        connection, client_address = sock.accept()

        # Receive the data in small chunks and retransmit it
        data = connection.recv(16)
        connection.close()
        sock.close()
        
        
        Y_new = np.zeros((1, state_dim))

        with open(final_pose_file, "r") as file:
            for line in file:
                vals = line.split()
                for i in xrange(0,state_dim):
                    Y_new[0,i] = vals[i]
        
#         print Y_new
        Y_train = np.concatenate((Y_train, Y_new),axis=0)
        model.set_XY(X=X_train, Y=Y_train)
        model.randomize()
        model.optimize('bfgs')
        
        #################### TEST
        test_filename = g_repo_path + '/data/' + 'push_data_action_'+ str(uncertainty)+'_1000_0.8_0.3_test.txt'
#         print test_filename
#         pos_err, quat_err,roll_error,pitch_error,yaw_error = test(test_filename, model)
        pos_err = test(test_filename, model)

#         print 'GP error: ', pos_err
        
        pos_err_NN = test_NN(test_filename, X_train, Y_train)
#         pos_err_NN, quat_err_NN,roll_error_NN,pitch_error_NN,yaw_error_NN = test_NN(test_filename, X_train, Y_train)
#         print 'NN error: ', pos_err_NN

#         test_err.append([pos_err, quat_err,roll_error,pitch_error,yaw_error ])
#         test_NN_err.append([pos_err_NN, quat_err_NN,roll_error_NN,pitch_error_NN,yaw_error_NN])
        test_err.append(pos_err[0])
        test_NN_err.append(pos_err_NN[0])
        
#         print count, X_train.shape, Y_train.shape
        count += 1
        elapsed_time = time.time() - start_time
        
    return model, X_train, Y_train, test_err, test_NN_err

In [5]:
def test(test_filename, model):
    obj_start_config_test, hand_start_config_test, obj_end_config_test, hand_end_config_test = \
                                                                                read_data(test_filename)
    X_test = np.squeeze(np.array(obj_start_config_test))
    Y_test = np.squeeze(np.array(obj_end_config_test))
    Y_predict = model.predict(X_test[:,:2])[0]
    gppred_pos_diff = np.abs(Y_predict[:,0:2] - Y_test[:,0:2])
#     ros_diff =  np.multiply(Y_test[:,3:], Y_predict[:,3:])**2
#     gppred_rot_diff = 1 - np.sum(ros_diff,axis=1)
    num_of_samples = X_test.shape[0]
    
    
#     roll_diff=[]
#     pitch_diff=[]
#     yaw_diff=[]

#     for i in range(0,len(X_test)):
#         angles = quaternion2rpy(Y_predict[i,3], Y_predict[i,4], Y_predict[i,5], Y_predict[i,6], form = 'Deg')
#         angles_gt = quaternion2rpy(Y_test[i,3], Y_test[i,4], Y_test[i,5], Y_test[i,6], form = 'Deg')
#         for rpy in angles:
#             if angles[rpy] > 180:
#                 angles[rpy] -= 360

#         for rpy in angles_gt:       
#             if angles_gt[rpy] > 180:
#                 angles_gt[rpy] -= 360
            
#     roll_diff.append(np.abs(angles['roll'] - angles_gt['roll']))
#     pitch_diff.append(np.abs(angles['pitch'] - angles_gt['pitch']))
#     yaw_diff.append(np.abs(angles['yaw'] - angles_gt['yaw']))
    

    
    return  np.sum(np.sqrt(np.sum(gppred_pos_diff**2,axis=1)))/num_of_samples, \
#             np.sum(gppred_rot_diff)/num_of_samples,\
#             np.sum(roll_diff)/num_of_samples,\
#             np.sum(pitch_diff)/num_of_samples,\
#             np.sum(yaw_diff)/num_of_samples
            

In [6]:
def test_NN(test_filename, X_train, Y_train):
    nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(X_train)

    obj_start_config_test, hand_start_config_test, obj_end_config_test, hand_end_config_test = \
                                                                                read_data(test_filename)
    X_test = np.squeeze(np.array(obj_start_config_test))
    Y_test = np.squeeze(np.array(obj_end_config_test))
    distances, indices = nbrs.kneighbors(X_test[:,:2])
#     print indices
    Y_predict = np.zeros((X_test.shape[0],2)) 
    for i in xrange(0,X_test.shape[0]):
        Y_predict[i,:] = Y_train[indices[i,0],:]
#     print Y_predict
#     Y_predict = model.predict(X_test)[0]
    gppred_pos_diff = np.abs(Y_predict[:,0:2] - Y_test[:,0:2])
#     ros_diff =  np.multiply(Y_test[:,3:], Y_predict[:,3:])**2
#     gppred_rot_diff = 1 - np.sum(ros_diff,axis=1)
    num_of_samples = X_test.shape[0]
    
#     roll_diff=[]
#     pitch_diff=[]
#     yaw_diff=[]
#     for i in range(0,len(X_test)):
#         angles = quaternion2rpy(Y_predict[i,3], Y_predict[i,4], Y_predict[i,5], Y_predict[i,6], form = 'Deg')
#         angles_gt = quaternion2rpy(Y_test[i,3], Y_test[i,4], Y_test[i,5], Y_test[i,6], form = 'Deg')
#         for rpy in angles:
#             if angles[rpy] > 180:
#                 angles[rpy] -= 360

#         for rpy in angles_gt:       
#             if angles_gt[rpy] > 180:
#                 angles_gt[rpy] -= 360
            
#     roll_diff.append(np.abs(angles['roll'] - angles_gt['roll']))
#     pitch_diff.append(np.abs(angles['pitch'] - angles_gt['pitch']))
#     yaw_diff.append(np.abs(angles['yaw'] - angles_gt['yaw']))
    
    return np.sum(np.sqrt(np.sum(gppred_pos_diff**2,axis=1)))/num_of_samples, \
#            np.sum(gppred_rot_diff)/num_of_samples, \
#             np.sum(roll_diff)/num_of_samples,\
#             np.sum(pitch_diff)/num_of_samples,\
#             np.sum(yaw_diff)/num_of_samples

In [7]:
# filename = '/home/zhusj/repos/blender_scene_generator/data/good_exp.txt'
# obj_start_config_list, hand_start_config_list, obj_end_config_list, hand_end_config_list, timelist\
#                                                         = read_raw_data(filename)
test_err_all =[]
test_NN_err_all =[]

all_model =[]
all_X_train=[]
all_Y_train=[]
total_trial = 100
for id in xrange(0,total_trial):  
#     print "starting Exp", id, " of ", len(obj_start_config_list)
# id = 0
    time_budget = 5.0

    X = np.array([0.016065, -0.039314])#, 0.763691, 0.999998, 0.001045, -0.001864, 0.000313])
    Y = np.array([0.020773, -0.059640])#, 0.763563, 0.995624, -0.001094, 0.000843, -0.093442])

    model, X_train, Y_train,test_err,test_NN_err = train(id, time_budget)
    
    print test_err
    print test_NN_err
    test_err_all.append(test_err)
    test_NN_err_all.append(test_NN_err)

#     all_model.append(model)
#     all_X_train.append(X_train)
#     all_Y_train.append(Y_train)
    
#     test_filename = g_repo_path + '/data/dynamics/test/' + 'push_data_action_' + str(id) +'_100_0.5_0.5_test.txt'
#     print test_filename
#     pos_err, quat_err,roll_error,pitch_error,yaw_error = test(test_filename, model)
#     print 'GP error: ', pos_err, quat_err,roll_error,pitch_error,yaw_error 
    
#     pos_err_NN, quat_err_NN,roll_error_NN,pitch_error_NN,yaw_error_NN = test_NN(test_filename, X_train, Y_train)
#     print 'NN error: ', pos_err_NN, quat_err_NN,roll_error_NN,pitch_error_NN,yaw_error_NN
    
#     test_err.append([pos_err, quat_err,roll_error,pitch_error,yaw_error ])
#     test_NN_err.append([pos_err_NN, quat_err_NN,roll_error_NN,pitch_error_NN,yaw_error_NN])


# print X.shape
     
#     train_filename = path + file
#     print train_filename
    

#     test_filename = current_path + 'test/' + 'push_data_action_' + str(id) +'_100_0.5_0.5_test.txt'
#     print test_filename
#     pos_err, quat_err = test(test_filename, model)
#     print 'error: ', pos_err, quat_err
    
#     raw_input("Press Enter to continue...")


#     test_err.append([pos_err, quat_err])

starting exp  0
X_train:  [[ 0.  0.]]
[[-0.04691  0.     ]] 0
new sample:  [[ 0.03927859 -0.03924817]]
new sample:  [[-0.03696091  0.03773473]]
new sample:  [[-0.03911538  0.03758631]]
new sample:  [[ 0.03916218 -0.03759943]]
new sample:  [[-0.03722788  0.0396475 ]]
new sample:  [[ 0.03513534 -0.03989356]]
new sample:  [[-0.03755293 -0.03854544]]
new sample:  [[ 0.03873819  0.03894145]]
new sample:  [[-0.03958171 -0.0374888 ]]
new sample:  [[ 0.02627418  0.01556655]]
new sample:  [[-0.03829492 -0.00222047]]
new sample:  [[-0.00093774  0.03948497]]
new sample:  [[-0.00308453 -0.03898797]]
new sample:  [[ 0.03965652  0.00201641]]
new sample:  [[ 0.03972912  0.03813776]]
new sample:  [[-0.03934654 -0.00660253]]
new sample:  [[-0.00287062 -0.03935366]]
new sample:  [[ 0.00592663  0.03988741]]
new sample:  [[ 0.03953951 -0.00528757]]
new sample:  [[-0.00885287  0.00287639]]
new sample:  [[-0.03903789 -0.03818242]]
new sample:  [[-0.03986035  0.00334252]]
new sample:  [[ 0.03943317  0.038877

In [396]:
max_number = 0
# total_trial = 10
for i in xrange(0,total_trial):  
#     print "current length:" , len(RS_result[i])
    if max_number < len(test_NN_err_all[i]):
        max_number = len(test_NN_err_all[i])
        print "max_number:", max_number

           
data = np.empty([total_trial,max_number])
for i in xrange(0,total_trial):    
    for j in xrange(0,len(test_NN_err_all[i])):
        data[i][j] = test_NN_err_all[i][j]

    for j in xrange(len(test_NN_err_all[i]), max_number):
        data[i][j] = data[i][j-1]
# data /=number_of_samples


dm = np.mean(data, axis=0)
ds = np.std(data, axis=0)

max_number: 23
max_number: 25


In [397]:
max_number = 0
# total_trial = 10
for i in xrange(0,total_trial):  
#     print "current length:" , len(RS_result[i])
    if max_number < len(test_err_all[i]):
        max_number = len(test_err_all[i])
        print "max_number:", max_number

           
data1 = np.empty([total_trial,max_number])
for i in xrange(0,total_trial):    
    for j in xrange(0,len(test_err_all[i])):
        data1[i][j] = test_err_all[i][j]

    for j in xrange(len(test_err_all[i]), max_number):
        data1[i][j] = data1[i][j-1]
# data /=number_of_samples


dm1 = np.mean(data1, axis=0)
ds1 = np.std(data1, axis=0)


max_number: 23
max_number: 25


In [398]:
dm

array([ 0.07551979,  0.06721953,  0.06157609,  0.05344779,  0.05219234,
        0.04865169,  0.04535406,  0.04024617,  0.03687875,  0.03361434,
        0.03170239,  0.02958141,  0.02754868,  0.02673734,  0.02590712,
        0.02551617,  0.02487073,  0.02439554,  0.02393894,  0.02377986,
        0.02363586,  0.02352101,  0.02348496,  0.02343164,  0.02340587])

In [399]:
dm1

array([ 0.05681311,  0.05445662,  0.0541759 ,  0.0534157 ,  0.05005301,
        0.0463387 ,  0.04189839,  0.03948306,  0.03561356,  0.03381238,
        0.03106691,  0.02934275,  0.02549075,  0.02369654,  0.02209085,
        0.02195204,  0.02003262,  0.02017953,  0.01934757,  0.01921924,
        0.01945407,  0.0192543 ,  0.0194133 ,  0.01957431,  0.01954917])

In [400]:
ds

array([ 0.00304667,  0.00449147,  0.00603583,  0.00774008,  0.00697428,
        0.00701063,  0.00785715,  0.00709221,  0.00782631,  0.00694923,
        0.00645808,  0.00466903,  0.00302727,  0.00206869,  0.00192666,
        0.00182748,  0.0018276 ,  0.00195646,  0.0018883 ,  0.00174555,
        0.00176655,  0.00162895,  0.00162213,  0.0016513 ,  0.00163679])

In [401]:
ds1

array([ 0.00492593,  0.00051854,  0.00167914,  0.00309594,  0.00539604,
        0.00614488,  0.00761536,  0.00817572,  0.00857101,  0.0096079 ,
        0.00933394,  0.00873012,  0.0073226 ,  0.00361277,  0.00280555,
        0.00281584,  0.0020846 ,  0.00242136,  0.00179074,  0.00160895,
        0.00174359,  0.00160287,  0.00167529,  0.00198044,  0.00198758])

In [131]:
ds1

array([ 0.00286221,  0.01116347,  0.01571783,  0.01598241,  0.015705  ,
        0.01697396,  0.01735195,  0.0178851 ,  0.01790867,  0.01797515,
        0.01801251,  0.01845835,  0.01864889,  0.01859892,  0.01882376,
        0.01911289,  0.01904518,  0.019112  ,  0.01821038,  0.01848119,
        0.01814086,  0.01817386,  0.01819879,  0.01830343,  0.01830798,
        0.01851934,  0.01816237,  0.01797057,  0.01780876,  0.01778787,
        0.01778787,  0.01778787])